In [1]:
import polars as pl
import duckdb
import utils_database as ud
import utils
import utils_model as um
%load_ext autoreload
%autoreload 2

In [8]:
# ud.start_session(user_id = 5,material_id=16)

In [3]:
# ud.clear_sessions()
# ud.clear_messages()
# len(ud.list_sessions(user_id=4))

In [2]:
df_suppliers = pl.read_excel(source="data/master_data.xlsx",sheet_name='Users')
df_materials = pl.read_excel(source="data/master_data.xlsx",sheet_name='Materials')
df_contracts = pl.read_excel(source="data/master_data.xlsx",sheet_name='Contracts')
df_sessions = pl.read_excel(source="data/master_data.xlsx",sheet_name='Sessions')

Could not determine dtype for column 7, falling back to string
Could not determine dtype for column 10, falling back to string


In [8]:
df_sessions.tail()

session_id,user_id,material_id,start_time,end_time
i64,i64,i64,date,date
9,4,12,2025-03-01,2025-03-01
10,4,13,2025-03-01,2025-03-01
11,5,14,2025-03-01,2025-03-01
12,5,15,2025-03-01,2025-03-01
13,5,16,2025-03-01,2025-03-01


In [4]:
# df_contracts = df_contracts.with_columns(
#     pl.struct([
#         'price_per_unit',
#         'quantity',
#         'bundling_unit',
#         'bundling_amount',
#         'bundling_discount',
#         'payment_term',
#         'payment_term_markup',
#         'delivery_timeline',
#         'contract_period',
#         'contract_inflation',
#         'rebates_threshold_unit',
#         'rebates_discount',
#         'warranty',
#         'incoterms'
#     ])
#     .map_elements(
#         lambda row: um.levers_to_offer_text(
#             price_per_unit = row['price_per_unit'],
#             quantity = row['quantity'],
#             bundling_unit = row['bundling_unit'],
#             bundling_amount = row['bundling_amount'],
#             bundling_discount = row['bundling_discount'],
#             payment_term = row['payment_term'],
#             payment_term_markup = row['payment_term_markup'],
#             delivery_timeline = row['delivery_timeline'],
#             contract_period = row['contract_period'],
#             contract_inflation = row['contract_inflation'],
#             rebates_threshold_unit = row['rebates_threshold_unit'],
#             rebates_discount = row['rebates_discount'],
#             warranty = row['warranty'],
#             incoterms = row['incoterms']
#         )
#     )
#     .alias('contract_details')
# )
# df_contracts.head()

In [5]:
df_materials.head(2)

material_id,description,quantity,price_per_unit,min_quantity,max_quantity,min_price_per_unit,max_price_per_unit
i64,str,i64,i64,i64,f64,i64,f64
1,"""Material 1""",14548,149,14548,18912.4,149,201.15
2,"""Material 2""",10144,104,10144,13187.2,104,140.4


In [3]:
df_materials = pl.read_excel(source="data/master_data.xlsx",sheet_name='Materials')
for material in df_materials.rows(named = True):
    material = ud.Material().load_from_data(material)
    ud.add_material(material=material)

In [4]:
df_suppliers = pl.read_excel(source="data/master_data.xlsx",sheet_name='Users')
for user in df_suppliers.rows(named = True):
    ud.add_user(username=user['username'],role=user['role'])

In [5]:
df_contracts = pl.read_excel(source="data/master_data.xlsx",sheet_name='Contracts')
for contract in df_contracts.rows(named = True):
    contract = ud.Contract().load_from_data(contract)
    ud.add_contract(contract=contract)

Could not determine dtype for column 7, falling back to string
Could not determine dtype for column 10, falling back to string


In [6]:
# ud.clear_sessions()
df_sessions = pl.read_excel(source="data/master_data.xlsx",sheet_name='Sessions')
for session in df_sessions.rows(named = True):
    session = ud.Session().load_from_data(session)
    ud.add_session(session=session)

In [7]:
con = duckdb.connect("chatbot.duckdb")
df = con.execute("""SELECT * FROM Sessions""").pl()
con.close()
df

session_id,user_id,material_id,start_time,end_time
i32,i32,i32,datetime[μs],datetime[μs]
1,2,2,2025-03-01 00:00:00,2025-03-01 00:00:00
2,2,3,2025-03-01 00:00:00,2025-03-01 00:00:00
3,2,4,2025-03-01 00:00:00,2025-03-01 00:00:00
4,2,1,2025-03-01 00:00:00,2025-03-01 00:00:00
5,3,8,2025-03-01 00:00:00,2025-03-01 00:00:00
…,…,…,…,…
9,4,12,2025-03-01 00:00:00,2025-03-01 00:00:00
10,4,13,2025-03-01 00:00:00,2025-03-01 00:00:00
11,5,14,2025-03-01 00:00:00,2025-03-01 00:00:00


In [3]:
con = duckdb.connect("chatbot.duckdb")
df = con.execute("""SELECT ss.session_id, u.username,m.description, ss.start_time, ss.end_time 
        FROM Sessions ss
        LEFT JOIN Users u on ss.user_id = u.user_id
        LEFT JOIN Materials m on ss.material_id = m.material_id
        order by ss.start_time desc""").pl()
con.close()
df

session_id,username,description,start_time,end_time
i32,str,str,datetime[μs],datetime[μs]
4,"""abc123""","""Material 1""",2025-03-01 00:00:00,2025-03-01 00:00:00
1,"""abc123""","""Material 2""",2025-03-01 00:00:00,2025-03-01 00:00:00
2,"""abc123""","""Material 3""",2025-03-01 00:00:00,2025-03-01 00:00:00
3,"""abc123""","""Material 4""",2025-03-01 00:00:00,2025-03-01 00:00:00
5,"""xyz321""","""Material 8""",2025-03-01 00:00:00,2025-03-01 00:00:00
6,"""xyz321""","""Material 9""",2025-03-01 00:00:00,2025-03-01 00:00:00
7,"""xyz321""","""Material 10""",2025-03-01 00:00:00,2025-03-01 00:00:00
8,"""cab312""","""Material 11""",2025-03-01 00:00:00,2025-03-01 00:00:00
9,"""cab312""","""Material 12""",2025-03-01 00:00:00,2025-03-01 00:00:00


In [13]:
ud.pull_sessions_by_user('monojit')

session_id,username,description,start_time,end_time
i32,str,str,datetime[μs],datetime[μs]
11,"""monojit""","""Material 14""",2025-03-01 00:00:00,2025-03-01 00:00:00
12,"""monojit""","""Material 15""",2025-03-01 00:00:00,2025-03-01 00:00:00
13,"""monojit""","""Material 16""",2025-03-01 00:00:00,2025-03-01 00:00:00


In [14]:
material = ud.get_material_info(material_id = 2)
contract = ud.get_last_contract(user_id=1,material_id=2)
MIN_LEVERS = {
    'price_per_unit':material['min_price_per_unit'],
    'quantity': material['min_quantity']
}
MAX_LEVERS = {
    'price_per_unit':material['max_price_per_unit'],
    'quantity': material['max_quantity']
}
contract

{'contract_id': 2,
 'user_id': 1,
 'material_id': 2,
 'contract_details': None,
 'price_per_unit': 104.0,
 'quantity': 10144.0,
 'bundling_unit': 20288.0,
 'bundling_amount': None,
 'bundling_discount': 5.0,
 'payment_term': 'NET60',
 'delivery_timeline': 17.0,
 'contract_period': 2.0,
 'contract_inflation': 1.0,
 'rebates_threshold_unit': 13085.0,
 'rebates_discount': 2.0,
 'warranty': 1.0,
 'incoterms': 'EXW',
 'created_at': datetime.datetime(2025, 3, 1, 10, 6, 11, 296000),
 'expiry_tmstp': datetime.datetime(2025, 3, 1, 10, 6, 11, 296000)}

In [ ]:
conn = duckdb.connect('chatbot.duckdb')
matrls = ud.list_materials(user_id=1,db_connection=conn)
conn.close()

[(4,), (1,), (3,), (2,), (5,)]